In [1]:
import sqlite3
import pandas as pd

# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

Create a table students with 5 sample rows

In [2]:
cursor.execute('''
CREATE TABLE students (
    id INTEGER,
    name TEXT,
    course TEXT,
    marks INTEGER,
    join_date TEXT
)
''')

cursor.executemany('''
INSERT INTO students VALUES (?, ?, ?, ?, ?)
''', [
    (1, 'Asha', 'AI', 85, '2022-01-10'),
    (2, 'Rahul', 'ML', 90, '2021-11-05'),
    (3, 'Priya', 'DS', 88, '2023-03-15'),
    (4, 'Kiran', 'AI', 92, '2020-07-20'),
    (5, 'Meena', 'ML', 80, '2022-06-30')
])

print("✅ Students table created with 5 rows.\n")

✅ Students table created with 5 rows.



Show all students from 'ML' course scoring more than 85

In [3]:
print("➡️ Students from ML course scoring > 85:")
df = pd.read_sql_query("SELECT * FROM students WHERE course='ML' AND marks>85", conn)
print(df, "\n")

➡️ Students from ML course scoring > 85:
   id   name course  marks   join_date
0   2  Rahul     ML     90  2021-11-05 



Show average marks per course using GROUP BY

In [4]:
print("➡️ Average marks per course:")
df = pd.read_sql_query("SELECT course, AVG(marks) AS avg_marks FROM students GROUP BY course", conn)
print(df, "\n")

➡️ Average marks per course:
  course  avg_marks
0     AI       88.5
1     DS       88.0
2     ML       85.0 



List students by highest marks using ORDER BY


In [5]:
print("➡️ Students ordered by highest marks:")
df = pd.read_sql_query("SELECT * FROM students ORDER BY marks DESC", conn)
print(df, "\n")

➡️ Students ordered by highest marks:
   id   name course  marks   join_date
0   4  Kiran     AI     92  2020-07-20
1   2  Rahul     ML     90  2021-11-05
2   3  Priya     DS     88  2023-03-15
3   1   Asha     AI     85  2022-01-10
4   5  Meena     ML     80  2022-06-30 



Show students who joined after Jan 1, 2022

In [6]:
print("➡️ Students who joined after 2022-01-01:")
df = pd.read_sql_query("SELECT * FROM students WHERE join_date > '2022-01-01'", conn)
print(df, "\n")

➡️ Students who joined after 2022-01-01:
   id   name course  marks   join_date
0   1   Asha     AI     85  2022-01-10
1   3  Priya     DS     88  2023-03-15
2   5  Meena     ML     80  2022-06-30 



 Create a second table projects and join with students

In [7]:
cursor.execute('''
CREATE TABLE projects (
    project_id INTEGER,
    project_name TEXT,
    student_id INTEGER
)
''')

cursor.executemany('''
INSERT INTO projects VALUES (?, ?, ?)
''', [
    (101, 'AI Chatbot', 1),
    (102, 'ML Model', 2),
    (103, 'Data Dashboard', 3),
    (104, 'AI Automation', 4),
    (105, 'ML Predictor', 5)
])

print("➡️ Students and their projects (INNER JOIN):")
df = pd.read_sql_query('''
SELECT s.name, s.course, p.project_name
FROM students s
JOIN projects p ON s.id = p.student_id
''', conn)
print(df, "\n")

➡️ Students and their projects (INNER JOIN):
    name course    project_name
0   Asha     AI      AI Chatbot
1  Rahul     ML        ML Model
2  Priya     DS  Data Dashboard
3  Kiran     AI   AI Automation
4  Meena     ML    ML Predictor 



INNER JOIN, LEFT JOIN, RIGHT JOIN, SELF JOIN examples

In [8]:
print("➡️ INNER JOIN:")
df = pd.read_sql_query('''
SELECT s.name, p.project_name
FROM students s
INNER JOIN projects p ON s.id = p.student_id
''', conn)
print(df, "\n")

print("➡️ LEFT JOIN:")
df = pd.read_sql_query('''
SELECT s.name, p.project_name
FROM students s
LEFT JOIN projects p ON s.id = p.student_id
''', conn)
print(df, "\n")

print("➡️ RIGHT JOIN (Simulated):")
df = pd.read_sql_query('''
SELECT p.project_name, s.name
FROM projects p
LEFT JOIN students s ON s.id = p.student_id
''', conn)
print(df, "\n")

print("➡️ SELF JOIN: Compare marks among students (previous/next-like logic)")
df = pd.read_sql_query('''
SELECT a.name AS student, a.marks AS current_marks, b.name AS other_student, b.marks AS compared_marks
FROM students a
JOIN students b ON a.id != b.id
WHERE a.id < 3
''', conn)
print(df, "\n")

➡️ INNER JOIN:
    name    project_name
0   Asha      AI Chatbot
1  Rahul        ML Model
2  Priya  Data Dashboard
3  Kiran   AI Automation
4  Meena    ML Predictor 

➡️ LEFT JOIN:
    name    project_name
0   Asha      AI Chatbot
1  Rahul        ML Model
2  Priya  Data Dashboard
3  Kiran   AI Automation
4  Meena    ML Predictor 

➡️ RIGHT JOIN (Simulated):
     project_name   name
0      AI Chatbot   Asha
1        ML Model  Rahul
2  Data Dashboard  Priya
3   AI Automation  Kiran
4    ML Predictor  Meena 

➡️ SELF JOIN: Compare marks among students (previous/next-like logic)
  student  current_marks other_student  compared_marks
0    Asha             85         Rahul              90
1    Asha             85         Priya              88
2    Asha             85         Kiran              92
3    Asha             85         Meena              80
4   Rahul             90          Asha              85
5   Rahul             90         Priya              88
6   Rahul             90         

Show names that appear more than once (duplicates)

In [9]:
cursor.executemany('INSERT INTO students VALUES (?, ?, ?, ?, ?)', [
    (6, 'Asha', 'AI', 91, '2023-05-10'),
])
print("➡️ Duplicate student names:")
df = pd.read_sql_query('''
SELECT name, COUNT(*) AS count
FROM students
GROUP BY name
HAVING COUNT(*) > 1
''', conn)
print(df, "\n")

➡️ Duplicate student names:
   name  count
0  Asha      2 



Convert course-wise monthly expenses (marks as expenses) into columns (Pivot)

In [10]:
print("➡️ Pivot (Course-wise marks):")
df = pd.read_sql_query('SELECT course, name, marks FROM students', conn)
pivot = df.pivot_table(index='name', columns='course', values='marks', aggfunc='first')
print(pivot, "\n")

➡️ Pivot (Course-wise marks):
course    AI    DS    ML
name                    
Asha    85.0   NaN   NaN
Kiran   92.0   NaN   NaN
Meena    NaN   NaN  80.0
Priya    NaN  88.0   NaN
Rahul    NaN   NaN  90.0 



Unpivot it back

In [11]:
print("➡️ Unpivoted data:")
unpivot = pivot.reset_index().melt(id_vars='name', var_name='course', value_name='marks')
print(unpivot, "\n")


➡️ Unpivoted data:
     name course  marks
0    Asha     AI   85.0
1   Kiran     AI   92.0
2   Meena     AI    NaN
3   Priya     AI    NaN
4   Rahul     AI    NaN
5    Asha     DS    NaN
6   Kiran     DS    NaN
7   Meena     DS    NaN
8   Priya     DS   88.0
9   Rahul     DS    NaN
10   Asha     ML    NaN
11  Kiran     ML    NaN
12  Meena     ML   80.0
13  Priya     ML    NaN
14  Rahul     ML   90.0 



Combine first and last names

In [12]:
cursor.execute("ALTER TABLE students ADD COLUMN last_name TEXT;")
cursor.execute("UPDATE students SET last_name='Singh' WHERE id IN (1,2,3,4,5,6)")
print("➡️ Full names after concatenation:")
df = pd.read_sql_query("SELECT name || ' ' || last_name AS full_name, course, marks FROM students", conn)
print(df)


➡️ Full names after concatenation:
     full_name course  marks
0   Asha Singh     AI     85
1  Rahul Singh     ML     90
2  Priya Singh     DS     88
3  Kiran Singh     AI     92
4  Meena Singh     ML     80
5   Asha Singh     AI     91
